[Homeへ戻る](Home.ipynb)

---

NumPyはPythonにおける高速数値計算用のライブラリである。型付き多次元配列オブジェクト (numpy.ndarray)や、数学関数などが使える。SciPyやmatplotlibなど、NumPyに基づくライブラリは多い

---

# 資料集

* [朱鷺の杜Wiki](http://ibisforest.org/index.php?python%2Fnumpy)
    - これ見ればよくね
* [NumPy 利用ノート](https://showa-yojyo.github.io/note/python-numpy.html)


# 高速化
NumPyの計算をより高速にするにはいくつかの方法がある
* BLAS, LAPACKを高速なものに取り換える
    - BLAS (Basic Linear Algebra Subprograms)は行列の積といった基本的な線形代数計算を高速で行うライブラリであり、主にFortranやCで実装されている（[BLASとは](http://azalea.s35.xrea.com/blas/whatIsBlas.html)）。様々な実装があり、IntelコンパイラについてくるIntel MKL (Math Kernel Library)が最強だが、Linux版以外は有料。無料ではGotoBLAS系列(GotoBLAS2、OpenBLAS)またはATLASが良く使われる。2014年時点ではATLASよりOpenBLASの方が速いっぽい（[OpenBLAS と ATLAS の性能を R 上で比較する](http://www.hnagata.net/archives/198)）
    - LAPACK (Linear Algebra PACKage)はBLASを呼び出して固有値・特異値問題といったより高度な線形代数計算を高速で行うライブラリ。LAPACK付属のBLASは遅いので、自分で用意した速いBLASに取り換えよう
    - これらはPython以前からある歴史の長いライブラリである。C/C++ではGSL(GNU Scientific Library)という科学計算用のライブラリを通じて使うことが多い
* Cythonを使う
    - [NumPy を扱う](http://omake.accense.com/static/doc-ja/cython/src/tutorial/numpy.html)
    - [NumPyとCythonを組み合わせると爆速！](http://kesin.hatenablog.com/entry/20120314/1331689014)
    - [[Python][Cython]メモ: numpyアクセス高速化](http://nekopuni.holy.jp/2014/10/pythoncython%E3%83%A1%E3%83%A2-numpy%E3%82%A2%E3%82%AF%E3%82%BB%E3%82%B9%E9%AB%98%E9%80%9F%E5%8C%96/)
* コードの書き方を工夫する

# BLASの導入
* [numpy・scipyでマルチスレッドBLAS・LAPACKを使う](http://qiita.com/_akisato/items/d5de60e38b5a69387bde) 
* [Numpy/Scipy with OpenBLAS for Ubuntu 12.04](http://osdf.github.io/blog/numpyscipy-with-openblas-for-ubuntu-1204.html)
* [Building NumPy and SciPy with GotoBLAS2](https://lucidfrontier45.wordpress.com/tag/blas/)
* [numpy, scipyちょっと遅い気がするって時はblas, lapack, atlasが入っているか見直しましょう](http://qiita.com/r9y9/items/b6b2a0ee00781f1bb480)

この辺はかなりCPUに近い低い階層の話なので、うまくいくかどうかはものすごい環境依存。時期や環境が違う人のやり方を真似してもうまくいかない
* OpenBLASを使ってみたかったので[Windows + 64bit Cygwin 環境で OpenBLAS をビルドしてみた](http://www.hnagata.net/archives/212#more-212)を参考にMinGW+OpenBLASでやったが失敗
    - MinGWをインストールし、`C:\MinGW\bin`のパスを通した。`C:\MinGW\msys\1.0\bin`のパスも通せという文献もあるが、中見たらCygwinとぶつかりそうな雰囲気を感じたのでやめておいた
    - OpenBLASは[tar版の最新版](github.com/xianyi/OpenBLAS/tarball/v0.2.11)をダウンロードしてmakeしようとして失敗
        + 後でやり直したらTARGET(CPUの種類)を指定しろと怒られていると分かったので、CPUの種類を調べ、`make TARGET=ATOM`と打ってみたらついに*成功*し、次のように表示された
```
  OS               ... CYGWIN_NT
  Architecture     ... x86_64
  BINARY           ... 64bit
  C compiler       ... GCC  (command line : gcc)
  Fortran compiler ... GFORTRAN  (command line : gfortran)
  Library Name     ... libopenblas_atomp-r0.2.11.a (Multi threaded; Max num-thre
ads is 2)
To install the library, you can run "make PREFIX=/path/to/your/installation inst
all".
```
    
* 昔からGSLを入れるときにお世話になっている金子さんのページの記事[Windows で GotoBLAS2 バージョン 1.13 と CBLAS をビルドとインストール（Windows 上の Cygwin を使用）](http://www.kkaneko.jp/rinkou/cygwin/cblas.html)を試してみたが失敗
    - コンパイラをCygwinのx86_64-w64-mingw32-gccとx86_64-w64-mingw32-gfortranに取り換えてやってみたがやはり失敗
* [公式](http://www.scipy.org/scipylib/building/windows.html)はATLASを使えと言っている
* Intelコンパイラを入れて付属のIntel MKLを使うのが最強っぽい

# MKLの導入
* [Building NumPy with the Intel Math Kernel Library (MKL)](http://www.scipy.org/scipylib/building/windows.html#building-numpy-with-the-intel-math-kernel-library-mkl)
* [Building From Source on Linux](http://www.scipy.org/scipylib/building/linux.html)
* [Building NumPy and SciPy with Intel Composer 2013 and the MKL](http://www.shocksolution.com/2013/01/building-numpy-and-scipy-with-intel-composer-2013-and-the-mkl/)
* [Ubuntu で Intel Math Kernel Library (マス・カーネル・ライブラリ) バージョン 11.1 Linux 版, Intel C++ Compiler (C++ コンパイラ) バージョン 2013 Linux 版をインストール](http://www.kkaneko.jp/rinkou/linux/mkllinux.html)
* [Numpy/Scipy with Intel® MKL](https://software.intel.com/en-us/articles/numpy-scipy-with-mkl)

Open BLAS2がうまく入った後に気づいたのだけど、Intelコンパイラ（および付属のIntel MKL）は学生版ならWindowsでも無料らしい([Intel Education Offerings](https://software.intel.com/en-us/intel-education-offerings/)→STUDENT)のでやっぱりMKL使おう
* Intel Parallel Studio XE 2015 Professional Edition XE for C++ Windowsってやつを入れるのだけど、Visual Studioを事前に入れておく必要があるっぽい。Visual Studio Professional 2013相当の無料版Visual Studio Community 2013ってやつが2014年11月に出てるのでそれを使おう
